## Battle of Neighborhood - The Preparation

### Import important Libraries that will be required entire Project
#### Here, we are considering to continue the complete project in same Notebook

In [149]:
# Import all required libraries
import numpy as np # To handle data in a vectorize manner

import pandas as pd # To get the data in DataFrame

import json # To get and read the JSON file

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # To convert an address into latitude and longitude values

import requests # To handle JSON requests

from pandas.io.json import json_normalize # To conver json file data into DataFrame

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans # To create cluster by K-Means 

!conda install -c conda-forge folium=0.5.0 --yes
import folium # Map rendering library

from bs4 import BeautifulSoup # Install and import BeautifulSoup4 for scraping the Wikipedia page and get the table in csv format

import lxml.html as lh
import re

print('Libraries imported')

Solving environment: done

# All requested packages already installed.

Libraries imported


### Read the table from Wikipedia page and get the table data into pandas DataFrame

In [2]:
# Read URL
input_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Fetch the content from the url, using the requests library
wiki_data = requests.get(input_url)

# Used 'lxml' parser to parse the url content and store it in a variable.
wiki_content = BeautifulSoup(wiki_data.content, "lxml")

table_data = wiki_content.find("table", class_="wikitable sortable")

# Create DataFrame
df_columns = []
df_rows = []

for i in table_data.find_all('th'):
    df_columns.append(re.split("\n", i.text)[0])

table_rows = table_data.find_all('tr')

for tr in table_rows:
    # Fetch Rows
    td = tr.find_all('td')
    row = [re.split("\n", tr.text)[0] for tr in td]
    df_rows.append(row)

PostalCode_df = pd.DataFrame(df_rows[1:],columns=df_columns)
PostalCode_df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


### Drop the rows with value as 'Not assigned' in Borough.
#### If Only Neighborhood is 'Not assigned', replace it with value in Borough column.

In [154]:
# Drop rows with "Not assigned"
T_df = PostalCode_df[~PostalCode_df['Borough'].isin(['Not assigned'])]
Toronto_df = pd.DataFrame(T_df.groupby(['Postcode', 'Borough'],sort=False)['Neighbourhood'].apply(','.join)).reset_index()
Toronto_df.rename(columns={'Postcode':'PostalCode','Neighbourhood':'Neighborhood'}, inplace=True)

for i, value in enumerate(Toronto_df['Neighborhood']):
    if value == 'Not assigned':
        Toronto_df['Neighborhood'][i] = Toronto_df['Borough'][i]
        
Toronto_df.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [4]:
# Display shape of the DataFrame
Toronto_df.shape

(103, 3)

### Collect location details i.e. latitude and longitude for each Postal Code

In [5]:
# Collect Latitude and Longitude data from 'http://cocl.us/Geospatial_data'
geo_data_url = 'http://cocl.us/Geospatial_data'

geo_data = pd.read_csv(geo_data_url)
geo_data.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
geo_data.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Add the Latitude and Longitude details to Postal Code DataFrame

In [6]:
# Merge the Neighbourhood DataFrame Geospatial data to get latitude and longitude info
Toronto_data = pd.merge(Toronto_df, geo_data, how='inner', on='PostalCode')
Toronto_data.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Not assigned,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


### Time to Explore the Neighborhood for the Postal Codes we have
#### Map all the Postal Code on Map and start exploring by visualizing 

In [7]:
# Explore and Cluster the Neighborhood

# Get latitude and longitude for Toronto City
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [83]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, postalcode in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Borough'], Toronto_data['PostalCode']):
    label = '{}, {}'.format(postalcode, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### In above map, we can see all the neighborhood across Toronto city. Let's try to see the Borough having word Toronto in it.

In [102]:
# Create DataFrame boroughs that contain the word Toronto
Toronto_data_only = Toronto_data[Toronto_data['Borough'].str.contains('Toronto')]
Toronto_data_only.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [103]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, postalcode in zip(Toronto_data_only['Latitude'], Toronto_data_only['Longitude'], Toronto_data_only['Borough'], Toronto_data_only['PostalCode']):
    label = '{}, {}'.format(postalcode, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Define Foursquare Credentials and Version

In [85]:
CLIENT_ID = 'OFKO1PZDZLJSWHGU4JCDGHV5RKLO5G3VGOOI5T1W1ABXHFKQ' # Foursquare ID
CLIENT_SECRET = '4324UQRBU2J3USV3KARAJRIPVCV2U5TMZ2UNGATC5JUBEMK1' # Foursquare Secret
VERSION = '20191016' # Foursquare API version

print('Foursquare credentials ready!')

Foursquare credentials ready!


### Explore Neighborhood in Toronto

#### To explore, we will take following steps:
1. Create a function to search venues for each Toronto locations
2. Create a DataFrame ('toronto_venues') with all the searched venues

In [128]:
# Get nearby Venues
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'PostalCode Latitude', 
                  'PostalCode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(names=Toronto_data_only['PostalCode'],
                                   latitudes=Toronto_data_only['Latitude'],
                                   longitudes=Toronto_data_only['Longitude']
                                  )

M5A
M5B
M5C
M4E
M5E
M5G
M6G
M5H
M6H
M5J
M6J
M4K
M5K
M6K
M4L
M5L
M4M
M4N
M5N
M4P
M5P
M6P
M4R
M5R
M6R
M4S
M5S
M6S
M4T
M5T
M4V
M5V
M4W
M5W
M4X
M5X
M4Y
M7Y


3. View Venues and get the shape and unique locations found

In [129]:
print(toronto_venues.shape)
toronto_venues.head()

(1709, 7)


,PostalCode,PostalCode Latitude,PostalCode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M5A,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,M5A,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,M5A,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [130]:
toronto_venues.groupby('PostalCode').count()
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 241 uniques categories.


### Analyze each Neighborhood

In [131]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['PostalCode'] = toronto_venues['PostalCode'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])

toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1709, 242)


,PostalCode,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [132]:
toronto_grouped = toronto_onehot.groupby('PostalCode').mean().reset_index()
toronto_grouped

,PostalCode,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4E,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.250000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,M4K,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023256,0.000000,...,0.023256,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.023256
2,M4L,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,M4M,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.051282,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.025641
4,M4N,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,M4P,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,M4R,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.045455
7,M4S,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027778,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
8,M4T,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,M4V,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,...,0.000000,0.00,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.00,0.000000


In [133]:
toronto_grouped.shape

(38, 242)

### Create DataFrame to display top 10 venues for each Postalcode

In [134]:
# Create function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [135]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postalcodes_venues_sorted = pd.DataFrame(columns=columns)
postalcodes_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    postalcodes_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

postalcodes_venues_sorted.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Health Food Store,Neighborhood,Pub,Trail,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
1,M4K,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Yoga Studio,Bookstore,Brewery,Bubble Tea Shop,Café
2,M4L,Park,Pet Store,Board Shop,Brewery,Burger Joint,Burrito Place,Sandwich Place,Pub,Pizza Place,Sushi Restaurant
3,M4M,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Chinese Restaurant,Bar,Stationery Store,Clothing Store,Middle Eastern Restaurant
4,M4N,Park,Swim School,Bus Line,Yoga Studio,Dog Run,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


### Cluster Neighborhood
#### Here, we will use KMeans Clustering method to create the clusters using the top 10 venues for each neighborhood

In [137]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [141]:
# add clustering labels
postalcodes_venues_sorted.drop('Cluster Labels', axis=1, inplace=True)
postalcodes_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = Toronto_data_only

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(postalcodes_venues_sorted.set_index('PostalCode'), on='PostalCode')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,0,Coffee Shop,Park,Café,Bakery,Pub,Breakfast Spot,Mexican Restaurant,Theater,Yoga Studio,Cosmetics Shop
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Middle Eastern Restaurant,Italian Restaurant,Pizza Place,Ice Cream Shop,Bubble Tea Shop,Bakery
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Restaurant,Hotel,Clothing Store,Cosmetics Shop,Italian Restaurant,Bakery,Breakfast Spot,Cocktail Bar
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Neighborhood,Pub,Trail,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Farmers Market,Seafood Restaurant,Beer Bar,Café,Steakhouse,Bakery,Cheese Shop,Fish Market


#### Let's visualize the resulting clusters

In [142]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters
Examine each cluster and determine the discriminating venue categories that distinguish each cluster

### Cluster 1

In [144]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,Coffee Shop,Park,Café,Bakery,Pub,Breakfast Spot,Mexican Restaurant,Theater,Yoga Studio,Cosmetics Shop
9,Downtown Toronto,0,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Middle Eastern Restaurant,Italian Restaurant,Pizza Place,Ice Cream Shop,Bubble Tea Shop,Bakery
15,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Hotel,Clothing Store,Cosmetics Shop,Italian Restaurant,Bakery,Breakfast Spot,Cocktail Bar
19,East Toronto,0,Health Food Store,Neighborhood,Pub,Trail,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
20,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Farmers Market,Seafood Restaurant,Beer Bar,Café,Steakhouse,Bakery,Cheese Shop,Fish Market
24,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Café,Ice Cream Shop,Burger Joint,Middle Eastern Restaurant,Sandwich Place,Bubble Tea Shop,Juice Bar,Bar
25,Downtown Toronto,0,Grocery Store,Café,Park,Restaurant,Coffee Shop,Baby Store,Italian Restaurant,Nightclub,Diner,Convenience Store
30,Downtown Toronto,0,Coffee Shop,Café,Steakhouse,Bar,Thai Restaurant,American Restaurant,Sushi Restaurant,Restaurant,Asian Restaurant,Burger Joint
31,West Toronto,0,Supermarket,Bakery,Pharmacy,Café,Bar,Bank,Brewery,Music Venue,Middle Eastern Restaurant,Art Gallery
36,Downtown Toronto,0,Coffee Shop,Hotel,Aquarium,Café,Scenic Lookout,Restaurant,Fried Chicken Joint,Italian Restaurant,Brewery,Bar


#### From above records we can see that, Cluster 1 has majorly having "Cafes, Restaurants and Pubs" in it's neighborhood

### Cluster 2

In [145]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,Central Toronto,1,Park,Playground,Yoga Studio,Discount Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
91,Downtown Toronto,1,Park,Playground,Building,Trail,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


#### From above records we can see that, Cluster 1 has majorly having "Parks and Playgrounds" in it's neighborhood

### Cluster 3

In [146]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Central Toronto,2,Park,Swim School,Bus Line,Yoga Studio,Dog Run,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


#### From above records we can see that, Cluster 1 has majorly having "Parks and Swim Schools" in it's neighborhood

### Cluster 4

In [147]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,3,Home Service,Garden,Yoga Studio,Dog Run,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


#### From above records we can see that, Cluster 1 has majorly having "Home Services and Gardens" in it's neighborhood

### Cluster 5

In [148]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Central Toronto,4,Trail,Jewelry Store,Park,Sushi Restaurant,Yoga Studio,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store


#### From above records we can see that, Cluster 1 has majorly having "Trails and Jewelry Stores" in it's neighborhood